In [512]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
nltk.download('stopwords');
nltk.download('punkt');



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/grahamarchibald/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/grahamarchibald/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [513]:
def multiple_word_remove_func(text, words_2_keep_list):
    '''
    Removes certain words from string, if present
    
    Step 1: Use word_tokenize() to get tokens from string
    Step 2: Removes the defined words from the created tokens
    
    Args:
        text (str): String to which the functions are to be applied, string
        words_2_remove_list (list): Words to be removed from the text, list of strings
    
    Returns:
        String with removed words
    '''     
    words_to_keep_list = words_2_keep_list
    
    words = word_tokenize(text)
    text = ' '.join([word for word in words if word in words_to_keep_list])
    return text


In [514]:
def most_freq_word_func(text, n_words=5):
    '''
    Returns the most frequently used words from a text
    
    Step 1: Use word_tokenize() to get tokens from string
    Step 2: Uses the FreqDist function to determine the word frequency
    
    Args:
        text (str): String to which the functions are to be applied, string
    
    Returns:
        List of the most frequently occurring words (by default = 5)
    ''' 
    words = word_tokenize(text)
    fdist = FreqDist(words) 
    
    df_fdist = pd.DataFrame({'Word': fdist.keys(),
                             'Frequency': fdist.values()})
    df_fdist = df_fdist.sort_values(by='Frequency', ascending=False)
    
    n_words = n_words
    most_freq_words_list = list(df_fdist['Word'][0:n_words])
    
    return most_freq_words_list

In [515]:
def most_rare_word_func(text, n_words=5):
    '''
    Returns the most rarely used words from a text
    
    Step 1: Use word_tokenize() to get tokens from string
    Step 2: Uses the FreqDist function to determine the word frequency
    
    Args:
        text (str): String to which the functions are to be applied, string
    
    Returns:
        List of the most rarely occurring words (by default = 5)
    ''' 
    words = word_tokenize(text)
    fdist = FreqDist(words) 
    
    df_fdist = pd.DataFrame({'Word': fdist.keys(),
                             'Frequency': fdist.values()})
    df_fdist = df_fdist.sort_values(by='Frequency', ascending=False)
    
    n_words = n_words
    # df_rare["Word"] = df_fdist['Word']
    # df_rare["Frequency"] = df_fdist['Frequency']
    most_rare_words_list = list(df_fdist['Word'][-n_words:])

    
    return most_rare_words_list, df_fdist #, df_rare

In [516]:
#generate dataframe from csv
df = pd.read_csv("/Users/grahamarchibald/Documents/SCHOOL/CPSC340/quaker/data/napa_earthquake_august-2014_subset_2.csv",
delimiter=";",
on_bad_lines='skip',
engine='python')

In [517]:
#remove numbers and special characters
df['tweet_text'] = df['tweet_text'].replace(to_replace=r"[^a-zA-Z0-9 ]",value="",regex=True)
#remove urls
df['tweet_text'] = df['tweet_text'].replace(to_replace=r"http\S+",value="",regex=True)
#convert all to lowercase
df['tweet_text'] = df['tweet_text'].str.lower()
#convert all to type string
df['tweet_text'] = df['tweet_text'].astype(str)
#remove words less than length four
df['tweet_text'] = df['tweet_text'].replace(to_replace=r"\b[a-zA-Z]\w{0,4}\b",value="",regex=True)
#remove 'stopwords'
stop = stopwords.words('english')
df['tweet_text_without_stopwords'] = df['tweet_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#remove all NA values
df['tweet_text_without_stopwords'].replace("",np.nan,inplace=True)
df.dropna(how='any',inplace=True)
#tokenize tweets
df['tokens'] = df["tweet_text_without_stopwords"].apply(nltk.word_tokenize)

df['tokens'].head(20)


0                                          [understand]
1                                           [professor]
2                              [anicasamonte, pieyucky]
3                        [thefunnyvines, evanperkins22]
5                                   [kreaytlin, modern]
6               [collapsing, vmworldproblems, ohmyshit]
7                                             [nervous]
8                                            [shopping]
9      [capping, perfect, wonderful, people, beautiful]
10                                             [angels]
11                       [burrrrr, jillians, francisco]
12    [rascal, flatts, shoreline, amphitheatre, moun...
13                                    [britney, spears]
14                                          [zeelmshah]
15                                    [realdrethegiant]
16                    [making, warmup, harder, thought]
17        [iiiiiiiiiiiiiiiiii, imagine, theres, closer]
19                                 [favoriting, 

In [518]:
#list of all words
all_words = ""
for idx, val in enumerate(df["tokens"]):
    for bal in val:
        all_words += " " + bal
#use this list as input to find frequency of all words
most_rare_words_list, df_rare = most_rare_word_func(all_words,n_words=len(all_words))

In [519]:
#use this list as input to find frequency of all words
# most_rare_words_list, df_rare = most_rare_word_func(all_words,n_words=len(all_words))

In [520]:
# test = "bigboy test earthquake fucking california untilnexttime"
# multiple_word_remove_func(test,words_to_keep)

In [521]:
# creating list of unique words, ie: unique words
words_to_rid = []
#set int(n) where n=frequency of word. ie: n=1 for unique words to be excluded. n=2 for unqique & words appearing twice etc..
df_rare['Frequency'][df_rare['Frequency']>int(5)] = np.nan
df_rare.dropna(how='any',inplace=True)
for idx, val in enumerate(df_rare["Word"]):
    words_to_rid.append(val)
words_to_rid;


/var/folders/tx/nbg81svj2rl4261_w_lgrxfh0000gn/T/ipykernel_86526/75082632.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rare['Frequency'][df_rare['Frequency']>int(5)] = np.nan


In [522]:
words_to_keep[0:100];

In [523]:
#generate dataframe from csv
df = pd.read_csv("/Users/grahamarchibald/Documents/SCHOOL/CPSC340/quaker/data/napa_earthquake_august-2014_subset_2.csv",
delimiter=";",
on_bad_lines='skip',
engine='python')
df = df

print("dataset length no processing : "+str(df.shape[0]))

#remove numbers and special characters
df['tweet_text'] = df['tweet_text'].replace(to_replace=r"[^a-zA-Z0-9 ]",value="",regex=True)
#remove urls
df['tweet_text'] = df['tweet_text'].replace(to_replace=r"http\S+",value="",regex=True)
#convert all to lowercase
df['tweet_text'] = df['tweet_text'].str.lower()
#convert all to type string
df['tweet_text'] = df['tweet_text'].astype(str)
#remove words less than length four
df['tweet_text'] = df['tweet_text'].replace(to_replace=r"\b[a-zA-Z]\w{0,4}\b",value="",regex=True)
#remove 'stopwords'
stop = stopwords.words('english')
df['tweet_text_without_stopwords'] = df['tweet_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#remove all NA values
df['tweet_text_without_stopwords'].replace("",np.nan,inplace=True)
df.dropna(how='any',inplace=True)

print("dataset length removing numbers, symbols, urls, words < 4 characters : "+str(df.shape[0]))

#remove all unique words
df['tweet_text_without_unqiue_words'] = df['tweet_text_without_stopwords'].apply(lambda x: ' '.join([word for word in x.split() if word not in (words_to_rid)]))
df['tweet_text_without_unqiue_words'].replace("",np.nan,inplace=True)
#remove new NA values
df.dropna(how='any',inplace=True)

print("dataset length removing unique words : "+str(df.shape[0]))

#tokenize tweets
df['tokens'] = df['tweet_text_without_unqiue_words'].apply(nltk.word_tokenize)

df['tokens'].head(20)

dataset length no processing : 42400
dataset length removing numbers, symbols, urls, words < 4 characters : 37414
dataset length removing unique words : 30077


0                                          [understand]
7                                             [nervous]
8                                            [shopping]
9               [perfect, wonderful, people, beautiful]
10                                             [angels]
11                                          [francisco]
12    [rascal, flatts, shoreline, amphitheatre, moun...
16                            [making, harder, thought]
17                            [imagine, theres, closer]
19                                             [tweets]
20                                 [bucket, winning, 3]
21                         [business, homeless, scared]
26                                    [selfie, country]
27                      [awesome, hiking, sanfrancisco]
29                                   [tonight, rolling]
31                                     [moment, decide]
32                                             [lounge]
33                                            [y

In [524]:
most_freq_word_func(all_words,n_words=50)

['earthquake',
 'california',
 'really',
 'francisco',
 'people',
 'thought',
 '60',
 '61',
 'everyone',
 'morning',
 'american',
 'fucking',
 'shaking',
 'canyon',
 'thanks',
 'tonight',
 'twitter',
 'pretty',
 'someone',
 'scared',
 '2',
 'magnitude',
 'always',
 'little',
 'earthquakes',
 'birthday',
 'tomorrow',
 'friends',
 'damage',
 'something',
 'school',
 'vote5sos',
 '3',
 'better',
 'getting',
 'anyone',
 'already',
 'nothing',
 'everything',
 'sanfrancisco',
 'asleep',
 'though',
 'theres',
 'literally',
 'actually',
 'oakland',
 'strong',
 'around',
 'happened',
 'talking']

In [10]:
# test.dropna(
#     axis=0,
#     how='any',
#     thresh=None,
#     subset=None,
#     inplace=True
# )
# #color_scale = [(0, 'orange'), (1,'red')]

# fig = px.scatter_mapbox(test, 
#                         lat="lat", 
#                         lon="lon", 
#                         hover_name="time", 
#                         hover_data=["time", "tweet_text"],
#                         zoom=8, 
#                         height=700,
#                         width=1244)

# fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()